## example of how to plot the grib output from ai-models

In [ ]:
import xarray as xr
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature

import metpy.calc as mpcalc
from metpy.units import units

import pickle
import os

### specify model name for plot titles, where your output files are, and where you want the images to go

In [ ]:
run_date = pd.Timestamp(2022,12,20,0)

#model_name = "Fourcastnet-v2"  ### for plot title
#model_abbrev = "fcnv2"  ### in filename

model_name = "Pangu-weather"  
model_abbrev = "pangu"

infiles = "output/output-"+model_abbrev+"-"+run_date.strftime("%Y%m%d")+"-*.grib"  ### path to output files, with wildcard for timestep

imagedir = "images"  ### where images will be written


### read in grib files 


#### need to filter different types of vertical levels, as cfgrib doesn't like reading them all in together

In [ ]:
ds_upper = xr.open_mfdataset(infiles, engine='cfgrib', indexpath='',
                        filter_by_keys={'typeOfLevel': 'isobaricInhPa'},
                            concat_dim='step', combine='nested').sortby('step')
ds_2m = xr.open_mfdataset(infiles, engine='cfgrib', indexpath='',
                        filter_by_keys={'typeOfLevel': 'heightAboveGround', 'level':2},
                            concat_dim='step', combine='nested').sortby('step')
ds_10m = xr.open_mfdataset(infiles, engine='cfgrib', indexpath='',
                        filter_by_keys={'typeOfLevel': 'heightAboveGround', 'level':10},
                            concat_dim='step', combine='nested').sortby('step')
ds_msl = xr.open_mfdataset(infiles, engine='cfgrib', indexpath='',
                        filter_by_keys={'typeOfLevel': 'meanSea'},
                            concat_dim='step', combine='nested').sortby('step')

init = pd.Timestamp(ds_msl.time.values)
print(init)

In [ ]:
ds_upper

### nice surface temperature colormap (optional); colormap for absolute vorticity

In [ ]:
### temperature from pickle
fp = open('sfc_temp_cmap.pkl', 'rb')
sfc_temp_cmap = pickle.load(fp)
fp.close()

### create 500avo colormap
colors1 = plt.cm.YlOrRd(np.linspace(0, 1, 36))
colors2 = plt.cm.BuPu(np.linspace(0.5, 0.75, 8))
colors_500avo = np.vstack((colors2, (1, 1, 1, 1), colors1))

## make some maps!

### setup

In [ ]:
datacrs = ccrs.PlateCarree()

plotcrs = ccrs.LambertConformal(central_longitude=-100, central_latitude=37.5)

def plot_background(ax):
     ### set up bounding box surrounding specified station
    latmin=27.75
    latmax=48.25
    lonmin=-118.7
    lonmax=-83.75

    ax.set_extent([lonmin,lonmax,latmin,latmax])
    ax.coastlines('50m', edgecolor='black', linewidth=0.9)
    ax.add_feature(cfeature.STATES.with_scale('10m'), linewidth=1.2)

    return lonmin,lonmax,latmin,latmax

### 2-m temperature, MSLP, 10-m winds

In [ ]:
#for tt in range(0,2):
for tt in range(0,len(ds_2m.step)):

    fig = plt.figure(figsize=(16,8.6))
    ax = fig.add_subplot(1,1,1,projection=plotcrs)

    lonmin, lonmax, latmin, latmax = plot_background(ax)

    vtime = pd.to_datetime(ds_10m.valid_time[tt].values)
    step = ds_10m.step[tt]
    
    mslp = ds_msl.msl.sel(step=step, longitude=slice(lonmin+360-10,lonmax+370), latitude=slice(latmax+5,latmin-5))
    u10 = ds_10m.u10.sel(step=step, longitude=slice(lonmin+360-10,lonmax+370), latitude=slice(latmax+5,latmin-5))
    v10 = ds_10m.v10.sel(step=step, longitude=slice(lonmin+360-10,lonmax+370), latitude=slice(latmax+5,latmin-5))
    t2m = ds_2m.t2m.sel(step=step, longitude=slice(lonmin+360-10,lonmax+370), latitude=slice(latmax+5,latmin-5))

    fcst_lead = int((vtime - init) / pd.Timedelta('1 hour'))

    print("lead "+str(fcst_lead)+", valid "+str(vtime))

    clevs = np.arange(-40,110,2)  ### range of temperatures

    lon2d, lat2d = np.meshgrid(mslp.longitude,mslp.latitude)

    ### MSLP
    cs1 = ax.contour(lon2d, lat2d,
                     mslp/100.,
                            np.arange(900,1100,4), colors='black',
                                 linewidths=2, transform=ccrs.PlateCarree(),
                     transform_first=True)
    plt.clabel(cs1, fontsize=11, inline=True, inline_spacing=5, fmt='%i',
                        rightside_up=True, use_clabeltext=True)

    ### 2-m temp
    cf = ax.contourf(lon2d, lat2d, 1.8*(t2m-273.15)+32.,
                         clevs,
                         cmap=sfc_temp_cmap, extend='both',
                         transform=ccrs.PlateCarree(), transform_first=True)
    cb = plt.colorbar(cf, orientation='horizontal', pad=0.01, aspect=50, shrink=0.6)
    cb.set_label('Temperature (F)', fontsize=10)
    cb.ax.tick_params(labelsize=10)
    cb.ax.locator_params(nbins=20)

    ### winds
    # Transform Vectors before plotting, then plot wind barbs.
    wind_slice = (slice(None, None, 4), slice(None, None, 4))
    ax.barbs(u10[wind_slice].longitude, u10[wind_slice].latitude,
             u10[wind_slice].values*1.943844, v10[wind_slice].values*1.943844,   ### to knots
             lw=0.9, length=5.5,
             transform=ccrs.PlateCarree())

    ax.set_title(model_name+"\nMSLP, 2-m temperature, 10-m winds",
                  loc="left", horizontalalignment='left', fontsize=10.5, fontweight='bold')
    ax.set_title("initialized "+init.strftime("%H%M UTC %a %d %b %Y")+"\n"+str(fcst_lead)+"-h forecast valid "+vtime.strftime("%H%M UTC %a %d %b %Y"),
                  loc="right", horizontalalignment='right', fontsize=9.5)

    outdir = imagedir+"/"+init.strftime("%Y%m%d%H")
    os.system("mkdir -p "+outdir)

    outfile = "t2m_"+model_abbrev+"_"+init.strftime("%Y%m%d%H")+"_f"+str(fcst_lead).zfill(3)+".png"

    plt.savefig(outdir+"/"+outfile,
                bbox_inches='tight', transparent=False, facecolor='white', dpi=225)

    plt.show()

    plt.close('all')

print("done!")

### 850-hPa temperature, heights, winds

In [ ]:
lev = 850.  ### in hPa

for tt in range(0,len(ds_upper.step)):

    fig = plt.figure(figsize=(16,8.6))
    ax = fig.add_subplot(1,1,1,projection=plotcrs)

    lonmin, lonmax, latmin, latmax = plot_background(ax)

    vtime = pd.to_datetime(ds_upper.valid_time[tt].values)
    step = ds_upper.step[tt]
    
    z = ds_upper.z.sel(step=step, longitude=slice(lonmin+360-10,lonmax+370), latitude=slice(latmax+5,latmin-5), isobaricInhPa=lev)
    u = ds_upper.u.sel(step=step, longitude=slice(lonmin+360-10,lonmax+370), latitude=slice(latmax+5,latmin-5), isobaricInhPa=lev)
    v = ds_upper.v.sel(step=step, longitude=slice(lonmin+360-10,lonmax+370), latitude=slice(latmax+5,latmin-5), isobaricInhPa=lev)
    t = ds_upper.t.sel(step=step, longitude=slice(lonmin+360-10,lonmax+370), latitude=slice(latmax+5,latmin-5), isobaricInhPa=lev)

    fcst_lead = int((vtime - init) / pd.Timedelta('1 hour'))

    print("lead "+str(fcst_lead)+", valid "+str(vtime))

    clevs = np.arange(-40,43,1)  ### range of temperatures

    lon2d, lat2d = np.meshgrid(z.longitude,z.latitude)

    ### height
    cs1 = ax.contour(lon2d, lat2d,
                     z/9.80665,
                        np.arange(0, 3001, 30), colors='black',
                                 linewidths=2, transform=ccrs.PlateCarree(),
                     transform_first=True)
    plt.clabel(cs1, fontsize=11, inline=True, inline_spacing=5, fmt='%i',
                        rightside_up=True, use_clabeltext=True)

    ### temp
    cf = ax.contourf(lon2d, lat2d, t-273.15,
                         clevs,
                         cmap=sfc_temp_cmap, extend='both',
                         transform=ccrs.PlateCarree(), transform_first=True)
    cb = plt.colorbar(cf, orientation='horizontal', pad=0.01, aspect=50, shrink=0.6)
    cb.set_label('Temperature (C)', fontsize=10)
    cb.ax.tick_params(labelsize=10)

    ### winds
    # Transform Vectors before plotting, then plot wind barbs.
    wind_slice = (slice(None, None, 4), slice(None, None, 4))
    ax.barbs(u[wind_slice].longitude, u[wind_slice].latitude,
             u[wind_slice].values*1.943844, v[wind_slice].values*1.943844,   ### to knots
             lw=0.9, length=5.5,
             transform=ccrs.PlateCarree())

    ax.set_title(model_name+"\n850-hPa temperature, heights, and winds",
                  loc="left", horizontalalignment='left', fontsize=10.5, fontweight='bold')
    ax.set_title("initialized "+init.strftime("%H%M UTC %a %d %b %Y")+"\n"+str(fcst_lead)+"-h forecast valid "+vtime.strftime("%H%M UTC %a %d %b %Y"),
                  loc="right", horizontalalignment='right', fontsize=9.5)

    outdir = imagedir+"/"+init.strftime("%Y%m%d%H")
    os.system("mkdir -p "+outdir)

    outfile = "850temp_"+model_abbrev+"_"+init.strftime("%Y%m%d%H")+"_f"+str(fcst_lead).zfill(3)+".png"

    plt.savefig(outdir+"/"+outfile,
                bbox_inches='tight', transparent=False, facecolor='white', dpi=225)

    plt.show()

    plt.close('all')

### 500-hPa absolute vorticity, heights, winds

In [ ]:
lev = 500.  ### in hPa

for tt in range(0,len(ds_upper.step)):

    fig = plt.figure(figsize=(16,8.6))
    ax = fig.add_subplot(1,1,1,projection=plotcrs)

    lonmin, lonmax, latmin, latmax = plot_background(ax)

    vtime = pd.to_datetime(ds_upper.valid_time[tt].values)
    step = ds_upper.step[tt]
    
    z = ds_upper.z.sel(step=step, longitude=slice(lonmin+360-10,lonmax+370), latitude=slice(latmax+5,latmin-5), isobaricInhPa=lev)
    u = ds_upper.u.sel(step=step, longitude=slice(lonmin+360-10,lonmax+370), latitude=slice(latmax+5,latmin-5), isobaricInhPa=lev)
    v = ds_upper.v.sel(step=step, longitude=slice(lonmin+360-10,lonmax+370), latitude=slice(latmax+5,latmin-5), isobaricInhPa=lev)

    fcst_lead = int((vtime - init) / pd.Timedelta('1 hour'))

    print("lead "+str(fcst_lead)+", valid "+str(vtime))

    clevs = list(range(-8, 1, 1))+list(range(12, 36, 1))

    lon2d, lat2d = np.meshgrid(z.longitude,z.latitude)

    ### height
    cs1 = ax.contour(lon2d, lat2d,
                     z/9.80665,
                        np.arange(4800, 6060, 60), colors='black',
                         linewidths=2, transform=ccrs.PlateCarree(),
                         transform_first=True)
    plt.clabel(cs1, fontsize=11, inline=True, inline_spacing=5, fmt='%i',
                        rightside_up=True, use_clabeltext=True)

    ### temp
    cf = ax.contourf(lon2d, lat2d,
                     1e5*mpcalc.absolute_vorticity(u, v),
                         clevs,
                         colors=colors_500avo, extend='both',
                         transform=ccrs.PlateCarree(), transform_first=True)
    cb = plt.colorbar(cf, orientation='horizontal', pad=0.01, aspect=50, shrink=0.6)
    cb.set_label('absolute vorticity ($*10^5$ s$^{-1}$)', fontsize=10)
    cb.ax.tick_params(labelsize=10)

    ### winds
    # Transform Vectors before plotting, then plot wind barbs.
    wind_slice = (slice(None, None, 4), slice(None, None, 4))
    ax.barbs(u[wind_slice].longitude, u[wind_slice].latitude,
             u[wind_slice].values*1.943844, v[wind_slice].values*1.943844,   ### to knots
             lw=0.9, length=5.5,
             transform=ccrs.PlateCarree())

    ax.set_title(model_name+"\n500-hPa absolute vorticity, heights, and winds",
                  loc="left", horizontalalignment='left', fontsize=10.5, fontweight='bold')
    ax.set_title("initialized "+init.strftime("%H%M UTC %a %d %b %Y")+"\n"+str(fcst_lead)+"-h forecast valid "+vtime.strftime("%H%M UTC %a %d %b %Y"),
                  loc="right", horizontalalignment='right', fontsize=9.5)

    outdir = imagedir+"/"+init.strftime("%Y%m%d%H")
    os.system("mkdir -p "+outdir)

    outfile = "500avo_"+model_abbrev+"_"+init.strftime("%Y%m%d%H")+"_f"+str(fcst_lead).zfill(3)+".png"

    plt.savefig(outdir+"/"+outfile,
                bbox_inches='tight', transparent=False, facecolor='white', dpi=225)

    plt.show()

    plt.close('all')